# Demonstrating true strength of FireDucks in Query planning and optimization

The [TPC-H](https://www.tpc.org/tpch/) is a decision support benchmark that consists of a suite of business-oriented ad-hoc queries and concurrent data modifications. We will use [Query-3](https://www.tpc.org/TPC_Documents_Current_Versions/pdf/TPC-H_v3.0.1.pdf#page=33) in this demonstration that deals with three large tables, namely `lineitem`, `customer`, and `orders` with complex join, gorupby, sort etc.

##  Preparing Data (Scale Factor: 10) 

In [1]:
!pip install -q -U fireducks pydantic pydantic_settings linetimer

In [2]:
!git clone https://github.com/fireducks-dev/polars-tpch.git

fatal: destination path 'polars-tpch' already exists and is not an empty directory.


In [3]:
!mkdir -p polars-tpch/data/tables/scale-10.0
!cd polars-tpch/tpch-dbgen; make; ./dbgen -s 10; mv *.tbl ../data/tables/scale-10.0

make: Nothing to be done for 'all'.
TPC-H Population Generator (Version 2.17.2)
Copyright Transaction Processing Performance Council 1994 - 2010


In [4]:
!ls polars-tpch/data/tables/*

customer.tbl  nation.tbl  part.tbl	region.tbl
lineitem.tbl  orders.tbl  partsupp.tbl	supplier.tbl


In [5]:
# generating parquet files with header from the above .tbl files
!cd polars-tpch; PATH_TABLES=data/tables SCALE_FACTOR=10.0 python -m scripts.prepare_data_pyarrow

Processing table: customer
Processing table: lineitem
Processing table: nation
Processing table: orders
Processing table: part
Processing table: partsupp
Processing table: region
Processing table: supplier


In [6]:
!ls polars-tpch/data/tables/*

customer.parquet  nation.parquet  part.parquet	    region.parquet
customer.tbl	  nation.tbl	  part.tbl	    region.tbl
lineitem.parquet  orders.parquet  partsupp.parquet  supplier.parquet
lineitem.tbl	  orders.tbl	  partsupp.tbl	    supplier.tbl


## Checking Evaluation Environments

In [7]:
import platform, psutil
print("="*30, "Evaluation Environment Information", "="*30)
print(f'platform: {platform.system()}')
print(f'architecture: {platform.machine()}')
print(f'processor: {platform.processor()}')
print(f'cpu: {psutil.cpu_count()}')

============================== Evaluation Environment Information ==============================
platform: Linux
architecture: x86_64
processor: x86_64
cpu: 48


In [8]:
!nvidia-smi

Mon Jan 13 16:09:00 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   52C    P0    42W / 250W |    810MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##  Defining Query

In [9]:
import os
import datetime
from linetimer import CodeTimer

# REF: https://www.tpc.org/TPC_Documents_Current_Versions/pdf/TPC-H_v3.0.1.pdf#page=33
def q3(datapath):
    m_name = getattr(pd.__spec__.loader, "fast_lib", pd.__name__).split(".")[0]
    
    # to avoid some incompatibility issues in cuDF pandas
    dt_cons = datetime.datetime if m_name == "cudf" else datetime.date

    with CodeTimer(name=f"Overall execution of q3 using {m_name}", unit="s"):    
        (
            pd.read_parquet(os.path.join(datapath, "customer.parquet"))
              .merge(pd.read_parquet(os.path.join(datapath, "orders.parquet")), 
                     left_on="c_custkey", right_on="o_custkey")
              .merge(pd.read_parquet(os.path.join(datapath, "lineitem.parquet")), 
                     left_on="o_orderkey", right_on="l_orderkey")
              .pipe(lambda df: df[df["c_mktsegment"] == "BUILDING"])
              .pipe(lambda df: df[df["o_orderdate"] < dt_cons(1995, 3, 15)])
              .pipe(lambda df: df[df["l_shipdate"] > dt_cons(1995, 3, 15)])
              .assign(revenue=lambda df: df["l_extendedprice"] * (1 - df["l_discount"]))
              .groupby(["l_orderkey", "o_orderdate", "o_shippriority"], as_index=False)
              .agg({"revenue": "sum"})[["l_orderkey", "revenue", "o_orderdate", "o_shippriority"]]
              .sort_values(["revenue", "o_orderdate"], ascending=[False, True])
              .reset_index(drop=True)            
              .head(10)
              .to_parquet(os.path.join(datapath, f"{m_name}_q3_result.parquet"))      
        )

In [10]:
import os
import datetime
from linetimer import CodeTimer

def optimized_q3(datapath):
    m_name = getattr(pd.__spec__.loader, "fast_lib", pd.__name__).split(".")[0]
    
    # to avoid some incompatibility issues in cuDF pandas    
    dt_cons = datetime.datetime if m_name == "cudf" else datetime.date

    with CodeTimer(name=f"Overall execution of optimized_q3 using {m_name}", unit="s"):
        # load only required columns from respective tables
        req_customer_cols = ["c_custkey", "c_mktsegment"] # (2/8)
        req_lineitem_cols = ["l_orderkey", "l_shipdate", "l_extendedprice", "l_discount"] #(4/16)
        req_orders_cols = ["o_custkey", "o_orderkey", "o_orderdate", "o_shippriority"] #(4/9)
        customer = pd.read_parquet(os.path.join(datapath, "customer.parquet"), columns = req_customer_cols)
        lineitem =  pd.read_parquet(os.path.join(datapath, "lineitem.parquet"), columns = req_lineitem_cols)
        orders =  pd.read_parquet(os.path.join(datapath, "orders.parquet"), columns = req_orders_cols)
    
        # advanced-filter: to reduce scope of “customer” table to be processed
        f_cust = customer[customer["c_mktsegment"] == "BUILDING"]

        # advanced-filter: to reduce scope of “orders” table to be processed
        f_ord = orders[orders["o_orderdate"] < dt_cons(1995, 3, 15)]

        # advanced-filter: to reduce scope of “lineitem” table to be processed
        f_litem = lineitem[lineitem["l_shipdate"] > dt_cons(1995, 3, 15)]

        (
            f_cust.merge(f_ord, left_on="c_custkey", right_on="o_custkey")
                  .merge(f_litem, left_on="o_orderkey", right_on="l_orderkey")
                  .assign(revenue=lambda df: df["l_extendedprice"] * (1 - df["l_discount"]))
                  .groupby(["l_orderkey", "o_orderdate", "o_shippriority"], as_index=False)
                  .agg({"revenue": "sum"})[["l_orderkey", "revenue", "o_orderdate", "o_shippriority"]]
                  .sort_values(["revenue", "o_orderdate"], ascending=[False, True])
                  .reset_index(drop=True)
                  .head(10)
                  .to_parquet(os.path.join(datapath, f"{m_name}_opt_q3_result.parquet"))
        )

In [11]:
datapath = "polars-tpch/data/tables/scale-10.0"

##  Native-pandas

In [12]:
import pandas as pd # native pandas

In [13]:
q3(datapath)

Code block 'Overall execution of q3 using pandas' took: 202.74181 s


In [14]:
optimized_q3(datapath)

Code block 'Overall execution of optimized_q3 using pandas' took: 17.28082 s


##  FireDucks-pandas

In [15]:
%load_ext fireducks.pandas  
import pandas as pd  # fireducks.pandas

In [16]:
q3(datapath)

Code block 'Overall execution of q3 using fireducks' took: 1.66185 s


In [17]:
optimized_q3(datapath)

Code block 'Overall execution of optimized_q3 using fireducks' took: 1.60413 s


In [18]:
%unload_ext fireducks.pandas

##  cuDF-pandas

In [19]:
%load_ext cudf.pandas
import pandas as pd # cudf pandas

In [20]:
q3(datapath)

Code block 'Overall execution of q3 using cudf' took: 30.82411 s


In [21]:
optimized_q3(datapath)

Code block 'Overall execution of optimized_q3 using cudf' took: 0.78262 s


##  Result-verification

In [22]:
r1 = pd.read_parquet(os.path.join(datapath, "pandas_q3_result.parquet"))
r2 = pd.read_parquet(os.path.join(datapath, "pandas_opt_q3_result.parquet"))
r3 = pd.read_parquet(os.path.join(datapath, "fireducks_q3_result.parquet"))
r4 = pd.read_parquet(os.path.join(datapath, "fireducks_opt_q3_result.parquet"))
r5 = pd.read_parquet(os.path.join(datapath, "cudf_q3_result.parquet"))
r6 = pd.read_parquet(os.path.join(datapath, "cudf_opt_q3_result.parquet"))

In [23]:
r1

,l_orderkey,revenue,o_orderdate,o_shippriority
0,4791171,440715.2185,1995-02-23,0
1,46678469,439855.3250,1995-01-27,0
2,23906758,432728.5737,1995-03-14,0
3,23861382,428739.1368,1995-03-09,0
4,59393639,426036.0662,1995-02-12,0
5,3355202,425100.6657,1995-03-04,0
6,9806272,425088.0568,1995-03-13,0
7,22810436,423231.9690,1995-01-02,0
8,16384100,421478.7294,1995-03-02,0
9,52974151,415367.1195,1995-02-05,0


In [24]:
r2

,l_orderkey,revenue,o_orderdate,o_shippriority
0,4791171,440715.2185,1995-02-23,0
1,46678469,439855.3250,1995-01-27,0
2,23906758,432728.5737,1995-03-14,0
3,23861382,428739.1368,1995-03-09,0
4,59393639,426036.0662,1995-02-12,0
5,3355202,425100.6657,1995-03-04,0
6,9806272,425088.0568,1995-03-13,0
7,22810436,423231.9690,1995-01-02,0
8,16384100,421478.7294,1995-03-02,0
9,52974151,415367.1195,1995-02-05,0


In [25]:
r3

,l_orderkey,revenue,o_orderdate,o_shippriority
0,4791171,440715.2185,1995-02-23,0
1,46678469,439855.3250,1995-01-27,0
2,23906758,432728.5737,1995-03-14,0
3,23861382,428739.1368,1995-03-09,0
4,59393639,426036.0662,1995-02-12,0
5,3355202,425100.6657,1995-03-04,0
6,9806272,425088.0568,1995-03-13,0
7,22810436,423231.9690,1995-01-02,0
8,16384100,421478.7294,1995-03-02,0
9,52974151,415367.1195,1995-02-05,0


In [26]:
r4

,l_orderkey,revenue,o_orderdate,o_shippriority
0,4791171,440715.2185,1995-02-23,0
1,46678469,439855.3250,1995-01-27,0
2,23906758,432728.5737,1995-03-14,0
3,23861382,428739.1368,1995-03-09,0
4,59393639,426036.0662,1995-02-12,0
5,3355202,425100.6657,1995-03-04,0
6,9806272,425088.0568,1995-03-13,0
7,22810436,423231.9690,1995-01-02,0
8,16384100,421478.7294,1995-03-02,0
9,52974151,415367.1195,1995-02-05,0


In [27]:
r5

,l_orderkey,revenue,o_orderdate,o_shippriority
0,4791171,440715.2185,1995-02-23,0
1,46678469,439855.3250,1995-01-27,0
2,23906758,432728.5737,1995-03-14,0
3,23861382,428739.1368,1995-03-09,0
4,59393639,426036.0662,1995-02-12,0
5,3355202,425100.6657,1995-03-04,0
6,9806272,425088.0568,1995-03-13,0
7,22810436,423231.9690,1995-01-02,0
8,16384100,421478.7294,1995-03-02,0
9,52974151,415367.1195,1995-02-05,0


In [28]:
r6

,l_orderkey,revenue,o_orderdate,o_shippriority
0,4791171,440715.2185,1995-02-23,0
1,46678469,439855.3250,1995-01-27,0
2,23906758,432728.5737,1995-03-14,0
3,23861382,428739.1368,1995-03-09,0
4,59393639,426036.0662,1995-02-12,0
5,3355202,425100.6657,1995-03-04,0
6,9806272,425088.0568,1995-03-13,0
7,22810436,423231.9690,1995-01-02,0
8,16384100,421478.7294,1995-03-02,0
9,52974151,415367.1195,1995-02-05,0


##  Cleanup

In [29]:
!rm -rf polars-tpch/data/tables